In [1]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit_algorithms.optimizers import COBYLA
import itertools
from collections import Counter

# 参数定义
n = 2  # 作业数量
m = 3  # 机器数量
N = int(np.ceil(np.log2(m)))  # 二进制位数表示机器号
l = 3  # C_max 的二进制位数
p = np.array([[1, 2, 3], [2, 3, 1]])  # 处理时间矩阵
b = np.array([[0, 0], [1, 0], [0, 1]])  # 机器的二进制表示 (0, 1, 2)
W = [2]  # 不可用机器集合（例如机器 2，对应二进制 [0, 1]）
K1 = 100  # 惩罚系数 1
P = 200  # 惩罚系数 2
num_qubits = n * N + l  # 总量子比特数


# 定义量子门函数
def append_multi_rzz_term(qc, qubits, gamma):
    if len(qubits) == 0:
        return
    if len(qubits) == 1:
        qc.rz(2 * gamma, qubits[0])
        return
    for q in qubits[:-1]:
        qc.cx(q, qubits[-1])
    qc.rz(2 * gamma, qubits[-1])
    for q in reversed(qubits[:-1]):
        qc.cx(q, qubits[-1])


def append__z_term(qc, qubit, gamma):
    qc.rz(2 * gamma, qubit)


def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def append__mixer_term(qc, qubit, beta):
    qc.rx(2 * beta, qubit)


# 成本电路 1：∑_{h=0}^{l-1} 2^h Z_h
def get_cost_circuit1(gamma, qc):
    for h in range(l):
        angle = gamma * 2 ** h
        append__z_term(qc, n * N + h, angle)
    return qc


# 成本电路 2：K1 ∑_{j∈W} ∑_{i=0}^{n-1} ∏_{k=0}^{N-1} (1/2 + (1/2 - b_{jk}) Z_{ik})
def get_cost_circuit2(gamma, qc):
    all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
    for j in W:
        for i in range(n):
            for combo in all_combinations:
                constant = 1.0 / (2 ** N)
                qubits = []
                for k in combo:
                    constant *= (1 - 2 * b[j, k])
                    qubits.append(i * N + k)
                append_multi_rzz_term(qc, qubits, K1 * constant * gamma)
    return qc


# 成本电路 3：P ∑_{j=0}^{m-1} (∑_{i=0}^{n-1} p_{ij} ∏_{k=0}^{N-1} (1/2 + (1/2 - b_{jk}) Z_{ik}) - ∑_{h=0}^{l-1} 2^h Z_h)^2
def get_cost_circuit3(gamma, qc):
    all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(0, N + 1))))

    # 展开平方项：(∑ C_j - C_max)^2 = ∑ C_j^2 - 2 ∑ C_j C_max + C_max^2
    for j in range(m):
        # C_j^2 项：∑_{i,i'} p_{ij} p_{i'j} ∏_{k} (...) ∏_{k'} (...)
        for i1 in range(n):
            for i2 in range(n):
                for combo1 in all_combinations:
                    for combo2 in all_combinations:
                        constant = (1 / (2 ** (2 * N))) * p[i1, j] * p[i2, j]
                        qubits = []
                        for k in combo1:
                            constant *= (1 - 2 * b[j, k])
                            qubits.append(i1 * N + k)
                        for k in combo2:
                            constant *= (1 - 2 * b[j, k])
                            qubits.append(i2 * N + k)
                        count = Counter(qubits)
                        qubits_final = [q for q, cnt in count.items() if cnt % 2 == 1]
                        if len(qubits_final) > 0:
                            append_multi_rzz_term(qc, qubits_final, P * constant * gamma)

        # -2 C_j C_max 项
        for i in range(n):
            for combo in all_combinations:
                constant = (1 / (2 ** N)) * p[i, j]
                qubits_prod = []
                for k in combo:
                    constant *= (1 - 2 * b[j, k])
                    qubits_prod.append(i * N + k)
                for h in range(l):
                    coef = constant * 2 ** h
                    qubits_zh = qubits_prod + [n * N + h]
                    if len(qubits_prod) > 0:
                        append_multi_rzz_term(qc, qubits_prod, -2 * P * coef * gamma)
                    if len(qubits_zh) > 0:
                        append_multi_rzz_term(qc, qubits_zh, 2 * P * coef * gamma)

    # C_max^2 项：∑_{h,h'} 2^h 2^{h'} Z_h Z_{h'}
    for h1 in range(l):
        for h2 in range(l):
            coef = 2 ** h1 * 2 ** h2 * P * gamma
            if h1 == h2:
                append__z_term(qc, n * N + h1, coef)  # Z_h^2 = I，实际为常数项
            else:
                append__zz_term(qc, n * N + h1, n * N + h2, coef)

    return qc


# 混合器电路
def get_mixer_circuit(beta, qc):
    for i in range(num_qubits):
        append__mixer_term(qc, i, beta)
    return qc


# 反转比特序
def invert_counts(s):
    return s[::-1]


# 成本函数
def cost_function(params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    for i in range(len(beta)):
        get_cost_circuit1(gamma[i], qc)
        get_cost_circuit2(gamma[i], qc)
        get_cost_circuit3(gamma[i], qc)
        get_mixer_circuit(beta[i], qc)
    qc.measure(range(num_qubits), range(num_qubits))

    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)

    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[n * N:]])
        s_ik = np.array([int(bit) for bit in bitstring[:n * N]]).reshape(n, N)

        # 第一项：C_max
        E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])

        # 第二项：惩罚不可用机器
        E2 = 0
        for j in W:
            for i in range(n):
                constant = 1
                for k in range(N):
                    constant *= (1 - (s_ik[i, k] - b[j, k]) ** 2)
                E2 += constant
        E2 = K1 * E2

        # 第三项：负载平衡惩罚
        E3 = 0
        for j in range(m):
            load = 0
            for i in range(n):
                constant = 1
                for k in range(N):
                    constant *= (1 - (s_ik[i, k] - b[j, k]) ** 2)
                load += p[i, j] * constant
            c_max = np.sum([2 ** h * z_h[h] for h in range(l)])
            E3 += P * (load - c_max) ** 2

        E = E1 + E2 + E3
        expectation += E * prob

    return expectation, counts


# 参数优化
def optimize_qaoa(params):
    def objective(params):
        expectation, _ = cost_function(params)
        return expectation

    optimizer = COBYLA(rhobeg=1.5, tol=1e-7)
    result = optimizer.minimize(fun=objective, x0=params)
    return result


# 插值法初始化函数
def interpolate_parameters(prev_params, p_old, p_new):
    if p_old == 0:
        return np.random.uniform(0, np.pi, p_new), np.random.uniform(0, 2 * np.pi, p_new)
    prev_beta = prev_params[:p_old]
    prev_gamma = prev_params[p_old:]
    new_beta = np.zeros(p_new)
    new_gamma = np.zeros(p_new)
    for i in range(p_new):
        t = i / (p_new - 1) if p_new > 1 else 0
        new_beta[i] = (1 - t) * prev_beta[0] + t * prev_beta[-1]
        new_gamma[i] = (1 - t) * prev_gamma[0] + t * prev_gamma[-1]
    return np.concatenate([new_beta, new_gamma])


# 主程序
import time

startTime = time.time()
min_energy = float('inf')
init_point = np.array([])
counts = {}
final_result = None

for p1 in range(1, 4):
    min_energy = 100000
    for k in range(10):
        print(f"第 {p1} 层，第 {k} 个参数")
        if k == 0 and p1 == 1:
            init_point_temp = np.concatenate([np.random.uniform(0, np.pi, p1),
                                              np.random.uniform(0, 2 * np.pi, p1)])
        elif k == 0:
            # 使用插值法从前一层生成新参数
            init_point_temp = interpolate_parameters(init_point, p1 - 1, p1)
        else:
            # 随机调整当前层的参数
            init_point_temp = init_point.copy()
            beta_p = np.random.uniform(0, np.pi)
            gamma_p = np.random.uniform(0, 2 * np.pi)
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
            init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
            init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)

        result = optimize_qaoa(init_point_temp)
        optimal_params = result.x
        energy, counts_temp = cost_function(result.x)

        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
            print(f"New best energy: {min_energy}")
            print(f"Optimal parameters: {init_point}")

endTime = time.time()
print(f"Total time: {endTime - startTime} seconds")
print(f"Final minimum energy: {min_energy}")
print(f"Final optimal parameters: {final_result.x}")
print(f"Final counts: {counts}")



第 1 层，第 0 个参数


C:\Users\李华宪\AppData\Local\Temp\ipykernel_63768\1448780747.py:137: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  backend = Aer.get_backend('qasm_simulator')
C:\Users\李华宪\AppData\Local\Temp\ipykernel_63768\1448780747.py:138: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  result = execute(qc, backend, seed_simulator=10, shots=10000).result()


New best energy: 4261.892399999999
Optimal parameters: [3.90804933 7.80302158]
第 1 层，第 1 个参数
第 1 层，第 2 个参数
第 1 层，第 3 个参数
New best energy: 1954.5282000000002
Optimal parameters: [5.69505669 4.54351996]
第 1 层，第 4 个参数
第 1 层，第 5 个参数
第 1 层，第 6 个参数
第 1 层，第 7 个参数
第 1 层，第 8 个参数
第 1 层，第 9 个参数
第 2 层，第 0 个参数
New best energy: 3465.3613000000014
Optimal parameters: [5.69501246 5.79124379 4.38260025 4.54354342]
第 2 层，第 1 个参数
New best energy: 2412.3080999999997
Optimal parameters: [7.14684392 1.50088635 4.59258592 1.26958147]
第 2 层，第 2 个参数
第 2 层，第 3 个参数
第 2 层，第 4 个参数
第 2 层，第 5 个参数
第 2 层，第 6 个参数
第 2 层，第 7 个参数
第 2 层，第 8 个参数
New best energy: 1844.421200000001
Optimal parameters: [8.64605294 2.59524501 4.59263729 3.71297691]
第 2 层，第 9 个参数
第 3 层，第 0 个参数
New best energy: 6339.4587999999985
Optimal parameters: [8.41821564 7.35713158 4.13949083 4.38652965 4.00379711 3.74906735]
第 3 层，第 1 个参数
New best energy: 2625.1574
Optimal parameters: [8.41828979 7.65108767 1.75674599 4.38646718 4.00362936 3.94888585]
第 3

In [ ]:
# 可视化最终电路
qc = QuantumCircuit(num_qubits, num_qubits)
qc.h(range(num_qubits))
for i in range(len(final_result.x) // 2):
    get_cost_circuit1(final_result.x[i + len(final_result.x) // 2], qc)
    get_cost_circuit2(final_result.x[i + len(final_result.x) // 2], qc)
    get_cost_circuit3(final_result.x[i + len(final_result.x) // 2], qc)
    get_mixer_circuit(final_result.x[i], qc)
qc.measure(range(num_qubits), range(num_qubits))
print(qc)